#### Installing necessary packages

In [1]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2023-03-04 15:22:33--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 172.217.203.136, 172.217.203.91, 172.217.203.190, ...
Connecting to dl.google.com (dl.google.com)|172.217.203.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14613 (14K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub’

linux_signing_key.p 100%[===================>]  14.27K  --.-KB/s    in 0s      

2023-03-04 15:22:33 (62.2 MB/s) - ‘linux_signing_key.pub’ saved [14613/14613]

OK
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:4 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [2015 B]
Get:5 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1082 B]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 https://pa

In [2]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2023-03-04 15:23:06--  http://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.217.193.128, 2607:f8b0:400c:c03::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.217.193.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7396711 (7.1M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   7.05M  --.-KB/s    in 0.04s   

2023-03-04 15:23:06 (169 MB/s) - ‘/tmp/chromedriver.zip’ saved [7396711/7396711]

Archive:  /tmp/chromedriver.zip
  inflating: /usr/local/bin/chromedriver  


In [3]:
# install selenium
!sudo apt install -y python3-selenium
!pip install selenium > /dev/null




The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 43 not upgraded.
Need to get 38.7 MB of archives.
After this operation, 174 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.1 [494 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.58+20.04 [37.9 MB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 c

In [4]:
# To check Google Chrome's version
!google-chrome --version

Google Chrome 110.0.5481.177 


In [5]:
# To check Chrome Driver's version
!chromedriver -v

ChromeDriver 110.0.5481.77 (65ed616c6e8ee3fe0ad64fe83796c020644d42af-refs/branch-heads/5481@{#839})


#### Importing necessary packages

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
# chrome_options.add_argument("--window-size=1920,1080")

In [7]:
import requests
import json
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
from bs4 import BeautifulSoup

#### Parameters

In [8]:
start = 0
offset = 50
no_of_urls_to_scrape = 1000

end = start + no_of_urls_to_scrape
print("Start: ", start, "\tEnd: ", end)

Start:  0 	End:  1000


In [9]:
driver = webdriver.Chrome(options = chrome_options)

#### Read test dataset

In [10]:
samples_count = 0
out_df = None
for chunk in pd.read_csv('/kaggle/input/telugu-news-urls/telangana_suryaa.csv', chunksize = no_of_urls_to_scrape):
    if samples_count == start:
        out_df = chunk.copy()
        break
    samples_count += int(chunk.shape[0])
    

if out_df is not None:
    print("Output Data:")
    print(out_df.shape)
    print(out_df.head())
else:
    exit('Something went wrong with reading the input data...')

Output Data:
(1000, 3)
   Unnamed: 0                                                url  sample_id
0           0  https://telangana.suryaa.com/telangana-updates...      41102
1           1  https://telangana.suryaa.com/telangana-updates...     125199
2           2  https://telangana.suryaa.com/telangana-updates...      10947
3           3  https://telangana.suryaa.com/telangana-updates...      30599
4           4  https://telangana.suryaa.com/telangana-updates...     127395


In [11]:
url = out_df['url'][0]
url

'https://telangana.suryaa.com/telangana-updates-2083-.html'

#### Scrape news content

In [12]:
def find_tag_sequence(element):
    parent_elements = []
    while element is not None:
        p_name = getattr(element, 'name', None)
        parent_elements.append(p_name)
        element = element.parent

        if element.parent is None:
            break
    return '>'.join(parent_elements[::-1])

In [13]:
def find_neighbours_and_parent(element):
    parent_tag = None
    prev_tag = None
    next_tag = None
    child_tags = None
    
    if element is not None:
        #### Next sibling
        next_sib = element.find_next_sibling()
        if next_sib is not None:
            next_tag = getattr(next_sib, 'name', None)
        
        #### Previous sibling
        previous_sib = element.find_previous_sibling()
        if previous_sib is not None:
            prev_tag = getattr(previous_sib, 'name', None)
        
        #### Element parent
        if element.parent is not None:
            parent = element.parent
            parent_tag = getattr(parent, 'name', None)
    
        #### Children
        childs = element.find_all(recursive=False)
        if childs is not None:
            child_tag_names = []
            for child in childs:
                child_tag_names.append(getattr(child, 'name', None))
            child_tags = list(set(child_tag_names.copy()))
    
    return {'next_sibling': next_tag, 'parent_node': parent_tag, 'prev_sibling': prev_tag, 'child_nodes': child_tags}

In [14]:
def extract_visual_features(web_element):
    features = {
        'x': np.nan,
        'y': np.nan,
        'height': np.nan,
        'width': np.nan,
        'is_visible': np.nan,
        'aria_role': np.nan,
        'bgcolor': np.nan,
        'font_color': np.nan,
        'font_size': np.nan,
        'font_style': np.nan,
        'font_family': np.nan,
    }
    
    
    try:
        loc = web_element.rect
        features['x'] = loc.get('x', np.nan)
        features['y'] = loc.get('y', np.nan)
        features['height'] = loc.get('height', np.nan)
        features['width'] = loc.get('width', np.nan)
    except:
        pass
    
    try:
        is_display = web_element.is_displayed()
        features['is_visible'] = is_display
    except:
        pass
    
    try:
        aria_role = web_element.aria_role
        features['aria_role'] = aria_role
    except:
        pass

    try:
        bgcolor = web_element.value_of_css_property('background-color')
        bgcolor = [int(val.strip()) for val in bgcolor.replace('rgba(','').replace(')','').split(',') if val.strip()!='']
        features['bgcolor'] = bgcolor
    except:
        pass
    
    try:
        font_size = web_element.value_of_css_property('font-size')
        features['font_size'] = font_size
    except:
        pass
    
    try:
        font_style = web_element.value_of_css_property('font-style')
        features['font_style'] = font_style
    except:
        pass
    
    try:
        font_family = web_element.value_of_css_property('font-family')
        features['font_family'] = font_family
    except:
        pass
    
    try:
        font_color = web_element.value_of_css_property('color')
        font_color = [int(val.strip()) for val in font_color.replace('rgba(','').replace(')','').split(',') if val.strip()!='']
        features['font_color'] = font_color
    except:
        pass
    
    
    return features

#### Extraction methods

In [15]:
def extract_title(soup, driver):
    try:
        #### Title Tag
        title = soup.title
        features = {}
        title_tag = np.nan
        if title is not None:
            title_tag = str(title)
            
            tag_sequence = ''
            try:
                tag_sequence = find_tag_sequence(title)
            except:
                pass
            if tag_sequence!='':
                features['tag_sequence'] = tag_sequence
            
            neighbours = {}
            try:
                neighbours = find_neighbours_and_parent(title)
            except:
                pass
            for key, value in neighbours.items():
                features[key] = value
                
            visual_features = {}
            try:
                if driver is not None:
                    title = driver.title
                    visual_features = extract_visual_features(title)
                    for key, value in visual_features.items():
                        features[key] = value
            except:
                pass
        else:
            title_tag = np.nan
            
        if len(features)>0:
            features = json.dumps(features, ensure_ascii=False)
        else:
            features = np.nan
        return title_tag, features
    except:
        return np.nan, np.nan

In [16]:
def extract_headline_category_pdate(soup, driver):
    try:
        #### Headline
        headline = np.nan
        headline_features = {}
        
        #### Category
        category = np.nan
        category_features = {}
        
        #### Publish Date
        publish_date = np.nan
        pdate_features = {}
        
        main_div = soup.find('div', attrs={'class':'container'})
        if main_div is not None:
            sub_div = main_div.find('div', attrs={'class':'container'})
            if sub_div is not None:
                content_div = sub_div.find('div', attrs={'class':'col-lg-12 col-md-12'})
                if content_div is not None:
                    h1 = content_div.find('h1')
                    if h1 is not None:
                        headline = str(h1)
                        
                        tag_sequence = ''
                        try:
                            tag_sequence = find_tag_sequence(h1)
                        except:
                            pass
                        if tag_sequence!='':
                            headline_features['tag_sequence'] = tag_sequence

                        try:
                            neighbours = find_neighbours_and_parent(h1)
                        except:
                            pass
                        for key, value in neighbours.items():
                            headline_features[key] = value
                            
                    spans = content_div.find_all('span')
                    if spans is not None and len(spans)>0:
                        cat = spans[0]
                        if cat is not None:
                            category = str(cat)
                            
                            tag_sequence = ''
                            try:
                                tag_sequence = find_tag_sequence(cat)
                            except:
                                pass
                            if tag_sequence!='':
                                category_features['tag_sequence'] = tag_sequence

                            try:
                                neighbours = find_neighbours_and_parent(cat)
                            except:
                                pass
                            for key, value in neighbours.items():
                                category_features[key] = value
                    
                        if len(spans)>1:
                            pdate = spans[1]
                            if pdate is not None:
                                publish_date = str(pdate)
                                
                                tag_sequence = ''
                                try:
                                    tag_sequence = find_tag_sequence(pdate)
                                except:
                                    pass
                                if tag_sequence!='':
                                    pdate_features['tag_sequence'] = tag_sequence

                                try:
                                    neighbours = find_neighbours_and_parent(pdate)
                                except:
                                    pass
                                for key, value in neighbours.items():
                                    pdate_features[key] = value
                                
                
            try:
                if driver is not None:
                    main_div_ = driver.find_element(By.CSS_SELECTOR, 'div.container')
                    if main_div_ is not None:
                        sub_div_ = main_div_.find_element(By.CSS_SELECTOR, 'div.container')
                        if sub_div_ is not None:
                            content_div_ = sub_div_.find_element(By.CSS_SELECTOR, 'div.col-lg-12.col-md-12')
                            if content_div_ is not None:
                                
                                ### Headline
                                h1_ = content_div_.find_element(By.TAG_NAME, 'h1')
                                if h1_ is not None:
                                    visual_features = extract_visual_features(h1_)
                                    for key, value in visual_features.items():
                                        headline_features[key] = value
                                        
                                spans_ = content_div.find_elements(By.TAG_NAME, 'span')
                                if spans_ is not None and len(spans_)>0:
                                    cat_ = spans_[0]
                                    if cat_ is not None:
                                        visual_features = extract_visual_features(cat_)
                                        for key, value in visual_features.items():
                                            category_features[key] = value
                                    
                                    if len(spans_)>1:
                                        pdate_ = spans_[1]
                                        if pdate_ is not None:
                                            visual_features = extract_visual_features(pdate_)
                                            for key, value in visual_features.items():
                                                pdate_features[key] = value
            except:
                pass
            
            headline = str(headline)
            category = str(category)
            publish_date = str(publish_date)
        else:
            headline = np.nan
            category = np.nan
            publish_date = np.nan
            
            
        if len(headline_features)>0:
            headline_features = json.dumps(headline_features, ensure_ascii=False)
        else:
            headline_features = np.nan
        
        if len(category_features)>0:
            category_features = json.dumps(category_features, ensure_ascii=False)
        else:
            category_features = np.nan
        
        if len(pdate_features)>0:
            pdate_features = json.dumps(pdate_features, ensure_ascii=False)
        else:
            pdate_features = np.nan
        return headline, headline_features, category, category_features, publish_date, pdate_features
    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

In [17]:
def extract_article_content(soup, driver):
    try:
        #### Article
        article_div = soup.find('div', attrs={'align': 'justify'})
        para_list = []
        features = []
        if article_div is not None:
            paras = article_div.find_all('p', attrs = {'class': ''})
            
            for i in range(len(paras)):
                features.append({})
            
            if paras is not None:
                count = 0
                    
                for para in paras:
                    para_list.append(str(para))
                    tag_sequence = ''
                    try:
                        tag_sequence = find_tag_sequence(para)
                    except:
                        pass
                    if tag_sequence!='':
                        features[count]['tag_sequence'] = tag_sequence
                    
                    neighbours = {}
                    try:
                        neighbours = find_neighbours_and_parent(para)
                    except:
                        pass
                    for key, value in neighbours.items():
                        features[count][key] = value
                    
                    count += 1
                
                try:
                    if driver is not None:
                        article_div_ = driver.find_element(By.XPATH, '//div[@align="justify"]')
                        if article_div_ is not None:
                            paras_ = article_div_.find_elements(By.TAG_NAME, 'p')
                            if paras_ is not None:
                                count = 0

                                for para_ in paras_:
                                    visual_features = {}
                                    try:
                                        visual_features = extract_visual_features(para_)
                                    except:
                                        pass
                                    for key, value in visual_features.items():
                                        features[count][key] = value

                                    count += 1
                except:
                    pass
            
            para_list = json.dumps(para_list, ensure_ascii=False)
        else:
            para_list = np.nan
            
        if len(features)>0:
            features = json.dumps(features, ensure_ascii=False)
        else:
            features = np.nan
        return para_list, features
    except:
        return np.nan, np.nan

In [18]:
def extract_images(soup, driver):
    try:
        image = None
        features = {}
        
        #### Images
        main_div = soup.find('div', attrs={'class':'container'})
        if main_div is not None:
            sub_div = main_div.find('div', attrs={'class':'container'})
            if sub_div is not None:
                content_div = sub_div.find_all('div', attrs={'class':'col-md-12'})
                if content_div is not None and len(content_div)>1:
                    content_div = content_div[1]
                    images = content_div.find('img')
                    if images is not None:
                        
                        image = str(images)
                        match = re.search(r'<img.*?>', image)
                        if match is not None:
                            image = image[match.span()[0]:match.span()[1]]
                        
                        tag_sequence = ''
                        try:
                            tag_sequence = find_tag_sequence(image)
                        except:
                            pass
                        if tag_sequence!='':
                            features['tag_sequence'] = tag_sequence

                        neighbours = {}
                        try:
                            neighbours = find_neighbours_and_parent(image)
                        except:
                            pass
                        for key, value in neighbours.items():
                            features[key] = value

            if driver is not None:
                main_div_ = driver.find_element(By.CSS_SELECTOR, 'div.container')
                if main_div_ is not None:
                    sub_div_ = main_div_.find_element(By.CSS_SELECTOR, 'div.container')
                    if sub_div_ is not None:
                        content_div_ = sub_div_.find_elements(By.CSS_SELECTOR, 'div.col-md-12')
                        if content_div_ is not None and len(content_div_)>1:
                            content_div_ = content_div_[1]
                            images_ = content_div_.find_element(By.TAG_NAME, 'img')
                            if images_ is not None:
                                visual_features = {}
                                try:
                                    visual_features = extract_visual_features(images_)
                                except:
                                    pass
                                for key, value in visual_features.items():
                                    features[key] = value

            image = str(image)
        else:
            image = np.nan
            
        if len(features)>0:
            features = json.dumps(features, ensure_ascii=False)
        else:
            features = np.nan
        
        return image, features
    except Exception as e:
#         print("Exception (img): ", e)
        return np.nan, np.nan

In [19]:
def extract_tags(soup, driver):
    try:
        #### Tags
        anchors = soup.find_all('a', attrs={'class': 'btn btn-sm btn-outline-secondary m-1'})
        tags_list = []
        features = []
        if anchors is not None:
            
            for count in range(len(anchors)):
                features.append({})
                
                anchor = anchors[count]
                tags_list.append(str(anchor))

                tag_sequence = ''
                try:
                    tag_sequence = find_tag_sequence(anchor)
                except:
                    pass
                if tag_sequence!='':
                    features[count]['tag_sequence'] = tag_sequence

                neighbours = {}
                try:
                    neighbours = find_neighbours_and_parent(anchor)
                except:
                    pass
                for key, value in neighbours.items():
                    features[count][key] = value

                    
            if driver is not None:
                anchors_ = driver.find_elements(By.CSS_SELECTOR, 'a.btn.btn-sm.btn-outline-secondary.m-1')
                if anchors_ is not None:
                    count = 0
                    for anchor_ in anchors_:

                        visual_features = {}
                        try:
                            visual_features = extract_visual_features(anchor_)
                        except:
                            pass
                        
                        for key, value in visual_features.items():
                            features[count][key] = value
                        count += 1
                                
            tags_list = json.dumps(tags_list, ensure_ascii=False)
        else:
            tags_list = np.nan
        
        
        if len(features)>0:
            features = json.dumps(features, ensure_ascii=False)
        else:
            features = np.nan    
        return tags_list, features
    except Exception as e:
#         print("Exception (tag): ", e)
        return np.nan, np.nan

In [20]:
def scrape_news_contents(source_code, url, sample_id, driver=None):
    soup = BeautifulSoup(source_code, 'html.parser')
    
    article = {
    	'url': url,
        'sample_id': sample_id,
        'title': np.nan,
        'title_features': np.nan,
        'headline': np.nan,
        'headline_features': np.nan,
        'category': np.nan,
        'category_features': np.nan,
        'publish_date': np.nan,
        'publish_date_features': np.nan,
        'article': np.nan,
        'article_features': np.nan,
        'images': np.nan,
        'images_features': np.nan,
        'tags': np.nan,
        'tags_features': np.nan,
        'source': soup,
    }

    title_tag, title_features = extract_title(soup, driver)
    article['title'] = title_tag
    article['title_features'] = title_features
    
    headline, headline_features, category, category_features, publish_date, date_features = extract_headline_category_pdate(soup, driver)
    article['headline'] = headline
    article['headline_features'] = headline_features
    
    article['category'] = category
    article['category_features'] = category_features
    
    article['publish_date'] = publish_date
    article['publish_date_features'] = date_features
    
    content, article_features = extract_article_content(soup, driver)
    article['article'] = content
    article['article_features'] = article_features
    
    images, images_features = extract_images(soup,driver)
    article['images'] = images
    article['images_features'] = images_features
    
    tags, tags_features = extract_tags(soup, driver)
    article['tags'] = tags
    article['tags_features'] = tags_features
    
    return article

In [21]:
def scrape_misc_contents(source_code, article, driver=None):
    misc_content = []
    misc_features = []
    
    try:
        ### Extracting the article features from the json to remove from misc content (leaf nodes)
        article_fields = []
        for field in SINGLE_ENTRIES:
            article_fields.append(article[field])

        for field in MULTIPLE_ENTRIES:
            entries = article[field]
            if type(entries)==str:
                entries = json.loads(entries)
                for entry in entries:
                    article_fields.append(entry)


        ### Extracting all leaf nodes from the soup object and removing the noisy tags, article fields
        all_leaf_nodes = []
        leaf_node_features = []
        soup = BeautifulSoup(source_code, 'html.parser')
        body = soup.find('body')
        for child in body.recursiveChildGenerator():
            name = getattr(child, "name", None)
            if name is None:
                if not child.isspace(): # leaf node, don't print spaces
                    p_name = getattr(child.parent, 'name', None)
                    if p_name is not None and p_name not in EXCLUDE_TAGS and str(child.parent) not in article_fields:
                        entry = {}
                        entry['misc_features'] = {}
                        entry['misc_features']['tag_sequence'] = find_tag_sequence(child.parent)
                        neighbours = find_neighbours_and_parent(child.parent)
                        for key, value in neighbours.items():
                            entry['misc_features'][key] = value

                        all_leaf_nodes.append(str(child.parent))
                        leaf_node_features.append(entry)


        ### Extracting the features for the leaf nodes (only for soup leaf nodes from above) using selenium
        leaf_node_selenium_features = {}
        if driver is not None:
            body_element = driver.find_element(By.TAG_NAME, 'body')
            if body_element is not None:
                childs = body_element.find_elements(By.XPATH,".//*")
                for child in childs:
                    try:
                        html_content = child.get_attribute('outerHTML')
                        if html_content in all_leaf_nodes:
                            index = all_leaf_nodes.index(html_content)
                            leaf_node_selenium_features[index] = extract_visual_features(child)
                    except:
                        pass


        ### Combining the features of soup and selenium
        for index, tag_html in enumerate(all_leaf_nodes):
            sel_feature = leaf_node_selenium_features.get(index, None)
            if sel_feature is not None:
                if len(sel_feature)>0:
                    for key, value in sel_feature.items():
                        leaf_node_features[index]['misc_features'][key] = value

            misc_content.append(all_leaf_nodes[index])
            misc_features.append(json.dumps(leaf_node_features[index].get('misc_features', {})))
    except Exception as e:
        print("Exception in misc: ", e)
        pass
    
    return misc_content, misc_features

#### Crawling the links

In [22]:
column_names = [
    'url', 'sample_id', 'title', 'title_features', 'headline',
    'headline_features', 'category', 'category_features', 'publish_date',
    'publish_date_features', 'article', 'article_features', 'images', 'images_features',
    'tags', 'tags_features', 'source'
]
column_names

['url',
 'sample_id',
 'title',
 'title_features',
 'headline',
 'headline_features',
 'category',
 'category_features',
 'publish_date',
 'publish_date_features',
 'article',
 'article_features',
 'images',
 'images_features',
 'tags',
 'tags_features',
 'source']

In [23]:
SINGLE_ENTRIES = ['title', 'headline', 'category', 'publish_date', 'images']
MULTIPLE_ENTRIES = ['article', 'tags']
EXCLUDE_TAGS = ['script', 'style', 'header', 'footer', 'iframe', 'formgroup', 'form']

In [24]:
output_file_name = './telangana_suryaa_{}_{}.csv'.format(start, end)
output_file_name_misc = './telangana_suryaa_{}_{}_misc.csv'.format(start, end)
output_file_name, output_file_name_misc

('./telangana_suryaa_0_1000.csv', './telangana_suryaa_0_1000_misc.csv')

In [25]:
print("Extracting the contents from the source")
final_data = []
final_misc_content = []
final_misc_features = []

for i in tqdm(range(start, start + min(end, out_df.shape[0]))):
    url = out_df['url'][i]
    sample_id = out_df['sample_id'][i]

    try:
        response = requests.get(url)
        if response.status_code==200:
            driver.get(url)
            article = scrape_news_contents(response.content, url, sample_id, driver)
            final_data.append(article)
            misc_content, misc_features = scrape_misc_contents(response.content, article, driver)
#             print("misc Len: ", len(misc_content))
#             print("misc Feat: ", len(misc_features))
            
            for i in range(len(misc_content)):
                if misc_content[i] not in final_misc_content:
                    final_misc_content.append(misc_content[i])
                    final_misc_features.append(misc_features[i])
#             print("Misc Len: ", len(final_misc_content))
#             print("Feat Len: ", len(final_misc_features))
            
        else:
            print("Status Code: ", response.status_code)
    except Exception as e:
        print("Exception: ", e)
        driver = webdriver.Chrome(options = chrome_options)


    if (i+1)%100 == 0:
        print("Saving the checkpoint at {}".format((i+1)/100))
        df = pd.DataFrame(final_data, columns=column_names)
        df.to_csv(output_file_name)
        
        misc_df = pd.DataFrame(data={'misc': final_misc_content, 'misc_features': final_misc_features}, columns=['misc', 'misc_features'])
        misc_df.to_csv(output_file_name_misc)

print("Saving the pandas data frame")
df = pd.DataFrame(final_data, columns=column_names)
df.to_csv(output_file_name)

misc_df = pd.DataFrame(data={'misc': final_misc_content, 'misc_features': final_misc_features}, columns=['misc', 'misc_features'])
misc_df.to_csv(output_file_name_misc)

Extracting the contents from the source


100%|██████████| 1000/1000 [5:11:59<00:00, 18.72s/it]


Saving the pandas data frame


In [26]:
df.head()

,url,sample_id,title,title_features,headline,headline_features,category,category_features,publish_date,publish_date_features,article,article_features,images,images_features,tags,tags_features,source
0,https://telangana.suryaa.com/telangana-updates...,41102,<title> జనహిత ప్రగతి సభలో మంత్రి కేటీఆర్</title>,"{""tag_sequence"": ""html>head>title"", ""next_sibl...","<h1 style=""font-family: 'Mallanna', sans-serif...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#c72c0d; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#333333; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","[""<p>నిజామాబాద్: పార్లమెంట్‌లో అద్బుతంగా మాట్ల...","[{""tag_sequence"": ""html>body>div>div>div>div>d...","<img alt="""" class=""img-responsive"" src=""https:...","{""x"": 107.5, ""y"": 373.1875, ""height"": 322, ""wi...","[""<a class=\""btn btn-sm btn-outline-secondary ...","[{""tag_sequence"": ""html>body>div>div>div>div>a...","[ , [\n (function(i,s,o,g,r,a,m){i['GoogleAna..."
1,https://telangana.suryaa.com/telangana-updates...,125199,<title> టెన్త్ పరీక్షల్లో కీలక మార్పులు...!\r\...,"{""tag_sequence"": ""html>head>title"", ""next_sibl...","<h1 style=""font-family: 'Mallanna', sans-serif...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#c72c0d; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#333333; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","[""<p>కరోనా కారణంగా ఈ ఏడాది కూడా విద్యారంగంపై త...","[{""tag_sequence"": ""html>body>div>div>div>div>d...","<img alt="""" class=""img-responsive"" src=""https:...","{""x"": 107.5, ""y"": 373.1875, ""height"": 305, ""wi...","[""<a class=\""btn btn-sm btn-outline-secondary ...","[{""tag_sequence"": ""html>body>div>div>div>div>a...","[ , [\n (function(i,s,o,g,r,a,m){i['GoogleAna..."
2,https://telangana.suryaa.com/telangana-updates...,10947,<title> ప్రభుత్వ ఉన్నతాధికారులకు హైకోర్టు ఆదేశ...,"{""tag_sequence"": ""html>head>title"", ""next_sibl...","<h1 style=""font-family: 'Mallanna', sans-serif...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#c72c0d; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#333333; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","[""<p style=\""text-align: justify;\""> ఈ నెల 7వ ...","[{""tag_sequence"": ""html>body>div>div>div>div>d...","<img alt="""" class=""img-responsive"" src=""https:...","{""x"": 107.5, ""y"": 373.1875, ""height"": 351, ""wi...","[""<a class=\""btn btn-sm btn-outline-secondary ...","[{""tag_sequence"": ""html>body>div>div>div>div>a...","[ , [\n (function(i,s,o,g,r,a,m){i['GoogleAna..."
3,https://telangana.suryaa.com/telangana-updates...,30599,<title> రేప్ కేసులో ఎమ్మెల్యేకు బిగుస్తున్న ఉచ...,"{""tag_sequence"": ""html>head>title"", ""next_sibl...","<h1 style=""font-family: 'Mallanna', sans-serif...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#c72c0d; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#333333; font-family:Arial,...","{""tag_sequence"": ""html>body>div>div>div>div>di...","[""<p>ఉత్తరప్రదేశ్‌లోని ఉన్నవ్ ఘటనపై యోగీ ఆదిత్...","[{""tag_sequence"": ""html>body>div>div>div>div>d...","<img alt="""" class=""img-responsive"" src=""https:...","{""x"": 107.5, ""y"": 373.1875, ""height"": 320, ""wi...","[""<a class=\""btn btn-sm btn-outline-secondary ...","[{""tag_sequence"": ""html>body>div>div>div>div>a...","[ , [\n (function(i,s,o,g,r,a,m){i['GoogleAna..."
4,https://telangana.suryaa.com/telangana-updates...,127395,<title> నందమూరి బాలకృష్ణ సంచలన వ్యాఖ్యలు</title>,"{""tag_sequence"": ""html>head>title"", ""next_sibl...","<h1 style=""font-family: 'Mallanna', sans-serif...","{""tag_sequence"": ""html>body>div>div>div>div>di...","<span style=""color:#c72c0d; font

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   url                    1000 non-null   object
 1   sample_id              1000 non-null   int64 
 2   title                  1000 non-null   object
 3   title_features         1000 non-null   object
 4   headline               1000 non-null   object
 5   headline_features      1000 non-null   object
 6   category               1000 non-null   object
 7   category_features      1000 non-null   object
 8   publish_date           1000 non-null   object
 9   publish_date_features  1000 non-null   object
 10  article                1000 non-null   object
 11  article_features       989 non-null    object
 12  images                 1000 non-null   object
 13  images_features        1000 non-null   object
 14  tags                   1000 non-null   object
 15  tags_features         

In [28]:
misc_df.head()

,misc,misc_features
0,<body>\n<!-- Navigation --><!-- Go to www.addt...,"{""tag_sequence"": ""html>body"", ""next_sibling"": ..."
1,"<div class=""bg-primary text-white text-center ...","{""tag_sequence"": ""html>body>div>div>div>div>di..."
2,"<a class=""text-secondary"" href=""https://telugu...","{""tag_sequence"": ""html>body>div>div>div>div>di..."
3,"<a class=""text-secondary"" href=""https://telugu...","{""tag_sequence"": ""html>body>div>div>div>div>di..."
4,"<a class=""text-secondary"" href=""https://telugu...","{""tag_sequence"": ""html>body>div>div>div>div>di..."


In [29]:
misc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4183 entries, 0 to 4182
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   misc           4183 non-null   object
 1   misc_features  4183 non-null   object
dtypes: object(2)
memory usage: 65.5+ KB


## Rough

In [30]:
# index = 2

# url = out_df['url'][index]
# sample_id = out_df['sample_id'][index]
# print("Sample ID: ", sample_id, "\tURL: ", url)

In [31]:
# print("Extracting the contents from the source")
# article = {}
# response = requests.get(url)
# if response.status_code==200:
#     driver.get(url)
#     article = scrape_news_contents(response.content, url, sample_id, driver)
# else:
#     print("Status Code: ", response.status_code)

In [32]:
# column_names = list(article.keys())
# column_names

In [33]:
# df = pd.DataFrame(data=[article], columns = column_names)
# df.shape

In [34]:
# df.head()